# Composite Analysis

This example highlights how material properties allow composite analysis in `sectionproperties`.

## Geometric vs. Composite

The default analysis type in `sectionproperties` is purely geometric, i.e. cross-section properties are reported based on the geometry only. In this analysis, all geometries are assigned the *default material*:

In [ ]:
from sectionproperties.pre.library import rectangular_section
from sectionproperties.analysis import Section

rect_geom = rectangular_section(d=100, b=50)
rect_geom.material

The default material has a unit elastic modulus, yield strength and density, and a Poisson's ratio of zero.

Geometric-only analysis allows geometric properties to be obtained:

In [ ]:
rect_geom.create_mesh(mesh_sizes=10)  # create mesh
rect_sec = Section(geometry=rect_geom)
rect_sec.calculate_geometric_properties()
ixx, iyy, ixy = rect_sec.get_ic()  # get second moments of area
print(f"Ixx = {ixx:.5e} mm4")

When a material property gets added to a geometry, the analysis becomes *composite*. 

In [ ]:
from sectionproperties.pre import Material

# assign steel to the geometry
steel = Material(
    name="Steel",
    elastic_modulus=200e3,
    poissons_ratio=0.3,
    density=7.85e-6,
    yield_strength=500,
    color="grey",
)
rect_geom.material = steel

# recreate mesh and section
rect_geom.create_mesh(mesh_sizes=10)
rect_sec = Section(geometry=rect_geom)
rect_sec.calculate_geometric_properties()

Cross-section properties are now modulus weighted as the assumption is that there are multiple regions with multiple different material properties. We can no longer obtain geometric-only properties:

In [ ]:
ixx, iyy, ixy = rect_sec.get_ic()  # get second moments of area

In this case, we need to get the modulus weighted second moments of area. Note we can still extract geometric properties by using a reference elastic modulus.

In [ ]:
# get modulus weighted second moments of area
eixx, eiyy, eixy = rect_sec.get_eic()
print(f"E.Ixx = {eixx:.5e} N.mm2")

# use reference elastic modulus to get transformed properties
ixx, iyy, ixy = rect_sec.get_eic(e_ref=steel)
print(f"Ixx = {ixx:.5e} mm4")

## Steel-Timber Composite Section

The following section models a composite timber floor and steel beam section. A universal steel beam (310UB40.4) is modelled with a 100D x 600W timber panel placed on its top flange.

### Create the Materials

In [ ]:
# create the steel material
steel = Material(
    name="Steel",
    elastic_modulus=200e3,
    poissons_ratio=0.3,
    density=7.85e-6,
    yield_strength=500,
    color="grey",
)

# create the timber material
timber = Material(
    name="Timber",
    elastic_modulus=8e3,
    poissons_ratio=0.35,
    yield_strength=20,
    density=0.78e-6,
    color="burlywood",
)

### Create the Geometry

In [ ]:
from sectionproperties.pre.library import i_section

# universal steel beam
ub = i_section(d=304, b=165, t_f=10.2, t_w=6.1, r=11.4, n_r=8, material=steel)

# timber floor panel
panel = rectangular_section(d=100, b=600, material=timber)
panel = panel.align_center(align_to=ub).align_to(other=ub, on="top")

# combine geometry
geom = ub + panel

## Create Mesh and Section Object

In [ ]:
# 10 mm2 mesh for UB, 500 mm2 mesh for timber
geom.create_mesh(mesh_sizes=[10, 500])
sec = Section(geometry=geom)
sec.plot_mesh()

### Perform Analysis

In [ ]:
sec.calculate_geometric_properties()
sec.calculate_warping_properties()
sec.calculate_plastic_properties()

### Display Analysis Results

We can plot the various centroids found by the analyses.

In [ ]:
sec.plot_centroids()

We can also print all the calculated section properties to the terminal, note that because have conducted a composite analysis, modulus weighted properties are displayed.

In [ ]:
sec.display_results()

We can also get transformed properties by specifying a reference material.

In [ ]:
ixx_timber, _, _ = sec.get_eic(e_ref=timber)
ixx_steel, _, _ = sec.get_eic(e_ref=steel)
print(f"Ixx,t = {ixx_timber:.3e} mm4")
print(f"Ixx,s = {ixx_steel:.3e} mm4")

A plastic analysis for composite sections will calculate plastic moments rather than plastic section moduli. The plastic moment assumes all geometry fibres reach the yield strength.

In [ ]:
mp_xx, _ = sec.get_mp()
print(f"Mp = {mp_xx / 1e6:.1f} kN.m")

### Stress Analysis

In [ ]:
stress = sec.calculate_stress(n=-100e3, mxx=-120e6, vy=-75e3)

In [ ]:
stress.plot_stress(stress="m_zz")

In [ ]:
stress.plot_stress(stress="vm")

We can plot only a specific list of materials by including the `material_list` argument. In the above plot it is difficult to see the stress in the timber so we set `material_list=[timber]`.

In [ ]:
stress.plot_stress(stress="vm", material_list=[timber])